In [ ]:
import os
import time

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import statistics
import csv
from scipy.optimize import minimize

from qiskit_optimization import QuadraticProgram
from qiskit.primitives import BackendSampler
from qiskit_algorithms.utils import algorithm_globals
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz

from mitiq import zne
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster
from collections import defaultdict

In [ ]:
bootqa_programs = ["gsdtsr","paintcontrol", "iofrol", "elevator", "elevator2"]
bootqa_programs_rep_values = {"gsdtsr":1,"paintcontrol":1,"iofrol":2, "elevator":4, "elevator2":4}
experiments = 10

In [ ]:
def get_data(data_name):
    """Read the datasets"""
    if data_name == "elevator":
        data = pd.read_csv("datasets/quantum_sota_datasets/elevator.csv", dtype={"cost": int, "input_div": float})
    elif data_name == "elevator2":
        data = pd.read_csv("datasets/quantum_sota_datasets/elevator.csv", dtype={"cost": int, "pcount": int, "dist": int})
    else:
        data = pd.read_csv("datasets/quantum_sota_datasets/" + data_name + ".csv", dtype={"time": float, "rate": float})
        data = data[data['rate'] > 0]
    return data

In [ ]:
bootqa_clusters = dict()

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    
    # Total suite metrics
    if bootqa_program == "elevator" or bootqa_program == "elevator2":
        test_cases_costs = data["cost"].tolist()
    else:
        test_cases_costs = data["time"].tolist()
    
    if bootqa_program == "elevator":
        test_cases_effectiveness = data["input_div"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite input div: {sum(test_cases_effectiveness)}")
    elif bootqa_program == "elevator2":
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite pcount: {sum(test_cases_pcount)}")
        print(f"Tot suite dist: {sum(test_cases_dist)}")
    else:
        test_cases_effectiveness = data["rate"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite rate: {sum(test_cases_effectiveness)}")
        
    # Normalize data
    if bootqa_program != "elevator2":
        cluster_data = np.column_stack((test_cases_costs, test_cases_effectiveness))
    else:
        cluster_data = np.column_stack((test_cases_costs, test_cases_pcount, test_cases_dist))
    
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(cluster_data)
    
    if bootqa_program == "elevator" or bootqa_program == "elevator2":
        num_clusters = 389
    if bootqa_program == "gsdtsr":
        num_clusters = 58
    if bootqa_program == "iofrol":
        num_clusters = 389
    if bootqa_program == "paintcontrol":
        num_clusters = 20
        
    max_cluster_dim = 5
    
    # Step 2: Perform K-Means Clustering
    start = time.time()
    linkage_matrix = linkage(normalized_data, method='ward')
    clusters = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')
    
    # Organize test cases by cluster
    clustered_data = defaultdict(list)
    for idx, cluster_id in enumerate(clusters):
        clustered_data[cluster_id].append(idx)
    
    # Process clusters to ensure none exceed max_cluster_dim
    new_cluster_id = max(clustered_data.keys()) + 1  # Start new IDs after existing ones
    to_add = []  # Collect new smaller clusters
    
    for cluster_id, elements in list(clustered_data.items()):  # Avoid modifying dict during iteration
        if len(elements) > max_cluster_dim:
            num_splits = -(-len(elements) // max_cluster_dim)  # Ceiling division to get the required number of splits
            split_size = -(-len(elements) // num_splits)  # Recalculate to distribute elements evenly
            
            # Split while keeping sizes balanced
            parts = [elements[i:i + split_size] for i in range(0, len(elements), split_size)]
    
            # Ensure all new clusters are within max_cluster_dim
            for part in parts:
                if len(part) > max_cluster_dim:
                    raise ValueError(f"A split cluster still exceeds max_cluster_dim ({len(part)} > {max_cluster_dim})!")
    
            # Add new parts to the new clusters
            to_add.extend(parts)
    
            # Remove original large cluster
            del clustered_data[cluster_id]
    
    # Assign new IDs to split parts
    for part in to_add:
        if part:  # Only add if the part is non-empty
            clustered_data[new_cluster_id] = part
            new_cluster_id += 1
    end = time.time()
    print("SelectQAOA Decomposition Time(ms): " + str((end-start)*1000))
    
    bootqa_clusters[bootqa_program] = clustered_data
    
    # Step 3: Calculate the metrics for each refined cluster
    cluster_metrics = {}
    for cluster_id, members in clustered_data.items():
        tot_cluster_costs = sum(test_cases_costs[i] for i in members)
        if bootqa_program != "elevator2":
            tot_cluster_effectiveness = sum(test_cases_effectiveness[i] for i in members)
        else:
            tot_cluster_pcount = sum(test_cases_pcount[i] for i in members)
            tot_cluster_dist = sum(test_cases_dist[i] for i in members)
        if bootqa_program != "elevator2":
            cluster_metrics[cluster_id] = {
                "tot_cluster_cost": tot_cluster_costs,
                "tot_cluster_rates": tot_cluster_effectiveness
            }
        else:
            cluster_metrics[cluster_id] = {
                "tot_cluster_cost": tot_cluster_costs,
                "tot_cluster_pcount": tot_cluster_pcount,
                "tot_cluster_dist": tot_cluster_dist
            }
        print(f"Cluster {cluster_id + 1} metrics:")
        print(f"Test Cases: {members}")
        print(f" - Num. Test Cases: {len(members):.2f}")
        print(f" - Execution Cost: {tot_cluster_costs:.2f}")
        if bootqa_program != "elevator2":
            print(f" - Failure Rate: {tot_cluster_effectiveness}")
        else:
            print(f" - PCount: {tot_cluster_pcount}")
            print(f" - Dist: {tot_cluster_dist}")
    
    print("===========================================================================")    
    
    for cluster_id in clustered_data.keys():
        if len(clustered_data[cluster_id]) > max_cluster_dim:
            print("Program: " + bootqa_program)
            print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    # Plotting the clusters in 3D space
    fig = plt.figure(figsize=(10, 7))
    if bootqa_program != "elevator2":
        ax = fig.add_subplot(111)
    else:
        ax = fig.add_subplot(111, projection='3d')
    
    # Extracting data for plotting
    exec_costs = np.array(test_cases_costs)
    if bootqa_program != "elevator2":
        effectiveness = np.array(test_cases_effectiveness)
    else:
        pcounts = np.array(test_cases_pcount)
        dists = np.array(test_cases_dist)
    
    # Plot each refined cluster with a different color
    colors = plt.cm.get_cmap("tab10", len(clustered_data))  # A colormap with as many colors as clusters
    for cluster_id, members in clustered_data.items():
        if bootqa_program != "elevator2":
            ax.scatter(
                exec_costs[members], 
                effectiveness[members], 
                color=colors(cluster_id % 10), 
                label=f"Cluster {cluster_id + 1}"
            )
        else:
            ax.scatter(
                exec_costs[members], 
                pcounts[members], 
                dists[members],
                color=colors(cluster_id % 10), 
                label=f"Cluster {cluster_id + 1}"
            )
    
    # Label the axes
    ax.set_xlabel("Execution Cost")
    if bootqa_program != "elevator2":
        ax.set_ylabel("Effectiveness")
    else:
        ax.set_ylabel("Passengers Count")
        ax.set_zlabel("Travel Distance")
    ax.legend()
    ax.set_title("Test Case Clustering Visualization for: " + bootqa_program)
    
    # Display the plot
    plt.show()


In [ ]:
def make_linear_terms_bootqa(cluster_test_cases, test_cases_costs, test_cases_rates, alpha):
    """Making the linear terms of the QUBO"""
    max_cost = max(test_cases_costs)
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append((alpha * ((test_cases_costs[test_case])/max_cost)) - ((1-alpha)*test_cases_rates[test_case]))
    
    return np.array(estimated_costs)

def make_linear_terms_bootqa2(cluster_test_cases, test_cases_costs, pcount, dist, alpha):
    """Making the linear terms of the QUBO for the elevator2 problem"""
    max_cost = max(test_cases_costs)
    max_pcount = max(pcount)
    max_dist = max(dist)
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append(((alpha/3) * ((test_cases_costs[test_case])/max_cost)) - ((alpha/3) * ((pcount[test_case])/max_pcount)) - ((alpha/3) * ((dist[test_case])/max_dist)))
    
    return np.array(estimated_costs)

In [ ]:
def create_linear_qubo(linear_terms):
    """This function is the one that has to encode the QUBO problem that QAOA will have to solve. The QUBO problem specifies the optimization to solve and a quadratic binary unconstrained problem"""
    qubo = QuadraticProgram()
    
    for i in range(0,len(linear_terms)):
        qubo.binary_var('x%s' % (i))

    qubo.minimize(linear=linear_terms)

    return qubo

In [ ]:
def bootstrap_confidence_interval(data, num_samples, confidence_alpha=0.95):
    """This function determines the statistical range within we would expect the mean value of execution times to fall; it relies on the bootstrapping strategy, which allows the calculation of the confidence interval by repeated sampling (with replacement) from the existing data to obtain an estimate of the confidence interval."""
    sample_means = []
    for _ in range(num_samples):
        bootstrap_sample = [random.choice(data) for _ in range(len(data))]
        sample_mean = np.mean(bootstrap_sample)
        sample_means.append(sample_mean)
    
    lower_percentile = (1 - confidence_alpha) / 2 * 100
    upper_percentile = (confidence_alpha + (1 - confidence_alpha) / 2) * 100
    lower_bound = np.percentile(sample_means, lower_percentile)
    upper_bound = np.percentile(sample_means, upper_percentile)
    
    return lower_bound, upper_bound

In [ ]:
bootqa_alphas = {"gsdtsr": 0.05,"paintcontrol": 0.55, "iofrol": 0.50, "elevator": 0.20, "elevator2": 5}

In [ ]:
def expectation_from_distribution(distribution, operator):
    """Calcola ⟨H⟩ da distribuzione e SparsePauliOp (solo Z/I)."""
    expectation = 0.0
    for pauli, coeff in zip(operator.paulis, operator.coeffs):
        pauli_str = pauli.to_label()
        if any(p in pauli_str for p in "XY"):  # ignora se contiene X o Y
            continue
        for bitstring, prob in distribution.items():
            if len(pauli_str) < len(bitstring):
                pauli_str = pauli_str.rjust(len(bitstring), "I")
            parity = 1
            for i in range(len(bitstring)):
                p = pauli_str[-(i + 1)]  # allineato da destra (qubit 0 a dx)
                if p == 'Z':
                    parity *= 1 if bitstring[-(i + 1)] == '0' else -1
            expectation += coeff.real * prob * parity
    return expectation.real

device_backend = FakeVigoV2()

# Config
run_times_dictionary = {k: [] for k in bootqa_alphas}
algorithm_globals.random_seed = 10598

scale_factors = [1.0, 3.0, 5.0]
factory = RichardsonFactory(scale_factors=scale_factors)

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    test_cases_costs = data["time"].tolist() if bootqa_program not in ["elevator", "elevator2"] else data["cost"].tolist()
    if bootqa_program == "elevator":
        test_cases_rates = data["input_div"].tolist()
    elif bootqa_program != "elevator2":
        test_cases_rates = data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()

    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []

    for i in range(experiments):
        final_selected_cases = []

        for cluster_id in bootqa_clusters[bootqa_program]:
            cluster = bootqa_clusters[bootqa_program][cluster_id]
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))

            # QUBO creation
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(cluster, test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(cluster, test_cases_costs, test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))

            # QAOA to build the circuit needs an Ising Hamiltonian as input
            # Since the problem is a QuadraticProgram object, we must convert it into an Ising
            qp2qubo = QuadraticProgramToQubo()
            qubo_problem = qp2qubo.convert(linear_qubo)
            hamiltonian, _ = qubo_problem.to_ising()
            backend = FakeVigoV2()

            # QAOAAnsatz builds a circuit with defined repetitions 
            reps = bootqa_programs_rep_values[bootqa_program]
            ansatz = QAOAAnsatz(hamiltonian, reps=reps)

            # Returns the 0 noise expected value of H with given params
            def zne_expectation(params):
                # Assigns params to γ and β (2*reps params)
                circuit = ansatz.assign_parameters(params)
                circuit.measure_all() # the circuit will produce a result in terms of bitstrings
            
                # Running the circuit on FakeVigo, getting the expected value for Hamiltonian 
                def run_expectation(circ):
                    sampler = BackendSampler(backend=device_backend)
                    result = sampler.run(circ).result()
                    dist = result.quasi_dists[0].binary_probabilities()
                    return expectation_from_distribution(dist, hamiltonian)
            
                # Running the the circuit at different noise levels, to extrapolate 0 noise H exp value
                factory = RichardsonFactory(scale_factors=[1.0, 3.0, 5.0])
                return zne.execute_with_zne(circuit, run_expectation, scale_noise=fold_global, factory=factory)
            
            # Initialize params
            initial_params = np.array([0.5] * ansatz.num_parameters)
            # Tries different params, calling for any trial zne_expectation, updates params based on H expected value retuned any time, until it converges
            result = minimize(zne_expectation, initial_params, method="COBYLA")
            
            # Get the optimal params
            optimal_params = result.x
                
            # Build the final circuit with the optimal params
            final_circuit = ansatz.assign_parameters(optimal_params)
            final_circuit.measure_all()
            
            # Execute the final circuite to get the final result
            sampler = BackendSampler(backend=device_backend)
            start = time.time()
            final_result = sampler.run(final_circuit).result()
            end = time.time()
            counts = final_result.quasi_dists[0].binary_probabilities()
            best_bitstring = max(counts.items(), key=lambda x: x[1])[0]
            
            # Ora puoi estrarre gli indici dei test selezionati
            selected_tests = [cluster[i] for i, b in enumerate(best_bitstring[::-1]) if b == "1"]
            print(f"[{bootqa_program}] Iter {i} - Best bitstring: {best_bitstring} -> {selected_tests}")

            for test in selected_tests:
                if test not in final_selected_cases:
                    final_selected_cases.append(test)

            run_times_dictionary[bootqa_program].append((end-start)*1000)

        cost = sum(test_cases_costs[t] for t in final_selected_cases)
        final_test_suite_costs.append(cost)

        if bootqa_program != "elevator2":
            effectiveness = sum(test_cases_rates[t] for t in final_selected_cases)
            final_effectivenesses.append(effectiveness)
        else:
            pcount = sum(test_cases_pcount[t] for t in final_selected_cases)
            dist = sum(test_cases_dist[t] for t in final_selected_cases)
            final_pcounts.append(pcount)
            final_dists.append(dist)

    avg_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    std_time = statistics.stdev(run_times_dictionary[bootqa_program]) 

    # Export results
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)",
                     "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, avg_time,
                  std_time, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)",
                     "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, avg_time,
                  std_time, run_times_dictionary[bootqa_program]]

    os.makedirs("results/selectqaoa/fake_vigo_zne", exist_ok=True)
    file_path = f"results/selectqaoa/fake_vigo_zne/{bootqa_program}-effectiveness-focus.csv"
    with open(file_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(var_names)
        writer.writerow(values)

    print(f"Results saved to {file_path}")
